<a href="https://colab.research.google.com/github/TitusChoi/Novelist_Classification/blob/master/Novelist_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Novelist_Classification.ipynb
Description   : 소설 작가 분류에 대한 예측 모델 구현<br>
Date : 2021.04.23 ~ 2021.04.29<br>
Author : 최유리, 정석영, 박민춘, 최디도

# 1. 필요 라이브러리 정의 및 데이터 목록 불러오기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import warnings 
warnings.filterwarnings(action='ignore')
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import seaborn as sns
import re
import random

In [ ]:
#경로 재설정
import os
os.chdir('./')

In [ ]:
#파일 불러오기
train = pd.read_csv('/content/drive/MyDrive/Novelist_Classification/datasets/train.csv', encoding = 'utf-8')
train_new = pd.read_csv('/content/drive/MyDrive/Novelist_Classification/datasets/new_train.csv', encoding = 'utf-8')
test = pd.read_csv('/content/drive/MyDrive/Novelist_Classification/datasets/test_x.csv', encoding = 'utf-8')
test_new = pd.read_csv('/content/drive/MyDrive/Novelist_Classification/datasets/new_test.csv', encoding = 'utf-8')
sample_submission = pd.read_csv('/content/drive/MyDrive/Novelist_Classification/datasets/sample_submission.csv', encoding = 'utf-8')
sample_submission_new = pd.read_csv('/content/drive/MyDrive/Novelist_Classification/datasets/new_sample_submission.csv', encoding = 'utf-8')

In [ ]:
#train 데이터 살펴보기
train

,index,text,author
0,0,"He was almost choking. There was so much, so m...",3
1,1,"“Your sister asked for it, I suppose?”",2
2,2,"She was engaged one day as she walked, in per...",1
3,3,"The captain was in the porch, keeping himself ...",4
4,4,"“Have mercy, gentlemen!” odin flung up his han...",3
...,...,...,...
54874,54874,"“Is that you, Mr. Smith?” odin whispered. “I h...",2
54875,54875,"I told my plan to the captain, and between us ...",4
54876,54876,"""Your sincere well-wisher, friend, and sister...",1
54877,54877,“Then you wanted me to lend you money?”,3


In [ ]:
train_new

,Unnamed: 0,text,author
0,0,It is hard to forget repulsive things. I remem...,6
1,1,It would be tedious if given in the beadle’s w...,7
2,2,“Very good. Shall we argue about it here in p...,2
3,3,“What! and I as high as a tree and as big as a...,6
4,4,"""Isn't it enough, Vanya?"" she cried, seeing hi...",3
...,...,...,...
83215,83215,"What with the birthday visitors, and what with...",7
83216,83216,It was an old rickety door and gave at once be...,2
83217,83217,"“Well then you can go to the devil,” said odin...",3
83218,83218,“Don’t know?”,7


In [ ]:
#test 데이터 살펴보기
del test_new['Unnamed: 0']
test_new

,index,text
0,0,"About thirty years ago Miss Maria Ward, of Hun..."
1,1,"Their homes were so distant, and the circles i..."
2,2,The letter was not unproductive. It re-establi...
3,3,"Such were its immediate effects, and within a ..."
4,4,Sir Thomas could not give so instantaneous and...
...,...,...
27313,27313,"At the end of another day or two, odin growing..."
27314,27314,"All afternoon we sat together, mostly in silen..."
27315,27315,"odin, having carried his thanks to odin, proc..."
27316,27316,"Soon after this, upon odin's leaving the room,..."


In [ ]:
#sample_submission
del sample_submission_new['Unnamed: 0']
sample_submission_new = sample_submission_new.drop(sample_submission_new.index[27318])
sample_submission_new

,index,0,1,2,3,4,5,6,7
0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,0,0
3,3,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
27313,27313,0,0,0,0,0,0,0,0
27314,27314,0,0,0,0,0,0,0,0
27315,27315,0,0,0,0,0,0,0,0
27316,27316,0,0,0,0,0,0,0,0


# 2. 데이터 전처리

## 2.1. 문장 부호를 제거

In [2]:
def alpha_num(text):
    return re.sub(r'[^A-Za-z0-9 ]', '', text)

train['text']=train['text'].apply(alpha_num)

NameError: ignored

## 2.2. 불용어 처리전 데이터 분리

In [3]:
# 불용어 제거해주는 함수
def remove_stopwords(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in base_stopwords:
            final_text.append(i.strip())
    return " ".join(final_text)

# 불용어
base_stopwords = [ "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", 
             "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", 
             "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", 
             "have", "having", "he", "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", 
             "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's", "its", "itself", 
             "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought", "our", "ours", 
             "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", "so", "some", "such", "than", "that", 
             "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll", 
             "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", "we", "we'd", "we'll", 
             "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom", 
             "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves" ]

In [4]:
#전처리 직접 적용
train['text'] = train['text'].str.lower()
test['text'] = test['text'].str.lower()
train['text'] = train['text'].apply(alpha_num).apply(remove_stopwords)
test['text'] = test['text'].apply(alpha_num).apply(remove_stopwords)


train_new['text'] = train_new['text'].str.lower()
test_new['text'] = test_new['text'].str.lower()
train_new['text'] = train_new['text'].apply(alpha_num).apply(remove_stopwords)
test_new['text'] = test_new['text'].apply(alpha_num).apply(remove_stopwords)

NameError: ignored

In [ ]:
# train test 분리
X_train = np.array([x for x in train['text']])
X_test = np.array([x for x in test['text']])
y_train = np.array([x for x in train['author']])

X_train_new = np.array([x for x in train_new['text']])
X_test_new = np.array([x for x in test_new['text']])
y_train_new = np.array([x for x in train_new['author']])

## 2.3. nltk 라이브러리 활용 불용어 제거

In [ ]:
# nltk 라이브러리를 사용한 불용어 제거
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')

nltk_stopword = set(stopwords.words('english'))

# Stopword 만 제거한 결과
def remove_stopwords_nltk(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in nltk_stopword:
            final_text.append(i.strip())
    return " ".join(final_text)

In [ ]:
# 단어 길이 확인
tmp = train

for i in range(len(tmp['text'])):
    tmp['text'][i] = tmp['text'][i].strip().split()

vocab = nltk.FreqDist(np.hstack(tmp['text']))
print('단어 집합의 크기 : {}'.format(len(vocab)))

## 2.4. 작가별 텍스트 시각화

In [ ]:
# 작가별 text 나누는 함수 정의
author_0 = train.loc[train['author'] == 0, 'text']
author_1 = train.loc[train['author'] == 1, 'text']
author_2 = train.loc[train['author'] == 2, 'text']
author_3 = train.loc[train['author'] == 3, 'text']
author_4 = train.loc[train['author'] == 4, 'text']
author_5 = train.loc[train['author'] == 5, 'text']
author_6 = train.loc[train['author'] == 6, 'text']
author_7 = train.loc[train['author'] == 7, 'text']

print(author_0.shape)
print(author_1.shape)
print(author_2.shape)
print(author_3.shape)
print(author_4.shape)
print(author_5.shape)
print(author_6.shape)
print(author_7.shape)

In [ ]:
# 작가별 text길이 시각화
sns.set_palette("Paired")
fig, axe = plt.subplots(ncols=1)
fig.set_size_inches(12,5)
sns.countplot(y_train,)
plt.show()

In [ ]:
# keras 문장 토큰화
## text_to_word_sequence는 알파벳을 소문자로 바꾸고, Punctuation을 제거한다
from tensorflow.keras.preprocessing.text import text_to_word_sequence


# 시리즈를 리스트로, 리스트를 문자열로 바꾸는 함수 (text_to_word_sequence가 문자열만 입력값으로 받기 때문)
def series_list_to_str(a):
    a.to_list()
    return " ".join([str(_) for _ in a])


str_author_0 = series_list_to_str(author_0)
str_author_1 = series_list_to_str(author_1)
str_author_2 = series_list_to_str(author_2)
str_author_3 = series_list_to_str(author_3)
str_author_4 = series_list_to_str(author_4)
str_author_5 = series_list_to_str(author_5)
str_author_6 = series_list_to_str(author_6)
str_author_7 = series_list_to_str(author_7)

In [ ]:
# 문장을 단어로 바꿔주는 함수
author_word_0 = text_to_word_sequence(str_author_0)
author_word_1 = text_to_word_sequence(str_author_1)
author_word_2 = text_to_word_sequence(str_author_2)
author_word_3 = text_to_word_sequence(str_author_3)
author_word_4 = text_to_word_sequence(str_author_4)
author_word_5 = text_to_word_sequence(str_author_5)
author_word_6 = text_to_word_sequence(str_author_6)
author_word_7 = text_to_word_sequence(str_author_7)
print(author_word_0[:20])

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
tokenizer = Tokenizer()

# fit_on_texts 적용 함수
# fit_on_texts()는 입력한 텍스트로부터 단어 빈도수가 높은 순으로 낮은 정수 인덱스를 부여


def fitontexts(words):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts([words])
    return tokenizer.word_index


# 이 과정으로 작가별 빈도수가 높은 단어를 구할 수 있음.
author0_index = fitontexts(author_word_0)
author1_index = fitontexts(author_word_1)
author2_index = fitontexts(author_word_2)
author3_index = fitontexts(author_word_3)
author4_index = fitontexts(author_word_4)
author5_index = fitontexts(author_word_5)
author6_index = fitontexts(author_word_6)
author7_index = fitontexts(author_word_7)



# print(author0_index)
# print(author1_index)
# print(author2_index)
# print(author3_index)
# print(author4_index)
# print(author5_index)
# print(author6_index)
# print(author7_index)

In [ ]:
# 상위 30개 많이 쓰인 단어
import seaborn as sns
from collections import Counter

fig = plt.figure(figsize=(17, 8))
rows = 2
cols = 4


def plot_top_non_stopwords_barchart(text):

    new = text.str.split()
    new = new.values.tolist()
    corpus = [word for i in new for word in i]

    counter = Counter(corpus)
    most = counter.most_common()
    x, y = [], []
    for word, count in most[:30]:
        x.append(word)
        y.append(count)

    return x, y


for i in range(8):
    x, y = plot_top_non_stopwords_barchart(
        train[train['author'] == i]['text'])
    ax = fig.add_subplot(rows, cols, i + 1)
    ax.set_title(i)
    sns.barplot(x=y, y=x)

In [ ]:
# 작가별 많이 쓴 단어 상위 10개
fig = plt.figure(figsize=(17, 8))  # rows*cols 행렬의 i번째 subplot 생성
rows = 2
cols = 4


def plot_top_non_stopwords_barchart2(text):

    new = text.str.split()
    new = new.values.tolist()
    corpus = [word for i in new for word in i]

    counter = Counter(corpus)
    most = counter.most_common()
    x, y = [], []
    for word, count in most[:10]:
        x.append(word)
        y.append(count)
    return x, y


for i in range(8):
    x, y = plot_top_non_stopwords_barchart2(
        train[train['author'] == i]['text'])
    ax = fig.add_subplot(rows, cols, i + 1)
    ax.set_title(i)
    sns.barplot(x=y, y=x)

### 2.4.1. N-gram 시각화

In [ ]:
from nltk.util import ngrams
from sklearn.feature_extraction.text import CountVectorizer
from collections import  Counter

def plot_top_ngrams_barchart(text, n=2):
    stop=set(base_stopwords)

    new= text.str.split()
    new=new.values.tolist()
    corpus=[word for i in new for word in i]

    def _get_top_ngram(corpus, n=None):
        vec = CountVectorizer(ngram_range=(n, n)).fit(corpus)
        bag_of_words = vec.transform(corpus)
        sum_words = bag_of_words.sum(axis=0) 
        words_freq = [(word, sum_words[0, idx]) 
                      for word, idx in vec.vocabulary_.items()]
        words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
        return words_freq[:10]

    top_n_bigrams=_get_top_ngram(text,n)[:10]
    x,y=map(list,zip(*top_n_bigrams))
    sns.barplot(x=y,y=x)

plot_top_ngrams_barchart(train['text'], n=3)    

In [ ]:
fig = plt.figure(figsize=(16,10)) # rows*cols 행렬의 i번째 subplot 생성
rows = 2
cols = 4

def plot_top_ngrams_barchart2(text, n=2):
    stop=set(base_stopwords)

    new= text.str.split()
    new=new.values.tolist()
    corpus=[word for i in new for word in i]

    def _get_top_ngram(corpus, n=None):
        vec = CountVectorizer(ngram_range=(n, n)).fit(corpus)
        bag_of_words = vec.transform(corpus)
        sum_words = bag_of_words.sum(axis=0) 
        words_freq = [(word, sum_words[0, idx]) 
                      for word, idx in vec.vocabulary_.items()]
        words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
        return words_freq[:10]

    top_n_bigrams=_get_top_ngram(text,n)[:10]
    x,y=map(list,zip(*top_n_bigrams))
    
    return x, y

for i in range(8):
    x,y = plot_top_ngrams_barchart2(train[train['author']==i]['text'], n=3)
    ax = fig.add_subplot(rows, cols, i+1)
    ax.set_title(i)
    sns.barplot(x=y,y=x)
    plt.tight_layout()

### 2.4.2. 토픽별 EDA

In [ ]:
!pip install pyLDAvis=='2.1.2'

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')

In [ ]:
# With NLTK you can tokenize and lemmatize easily:

import numpy as np
import nltk
from nltk.corpus import stopwords
import gensim
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.tokenize import word_tokenize
import pyLDAvis.gensim

NUM_TOPICS = 8


def get_lda_objects(text):
    nltk.download('stopwords')
    stop = set(base_stopwords)

    def _preprocess_text(text):
        corpus = []
        stem = PorterStemmer()
        lem = WordNetLemmatizer()
        for news in text:
            words = [w for w in word_tokenize(news) if (w not in stop)]

            words = [lem.lemmatize(w) for w in words if len(w) > 2]

            corpus.append(words)
        return corpus

    corpus = _preprocess_text(text)

    dic = gensim.corpora.Dictionary(corpus)
    bow_corpus = [dic.doc2bow(doc) for doc in corpus]

    lda_model = gensim.models.LdaMulticore(bow_corpus,
                                           num_topics=NUM_TOPICS,
                                           id2word=dic,
                                           passes=10,
                                           workers=2)

    return lda_model, bow_corpus, dic


def plot_lda_vis(lda_model, bow_corpus, dic):
    pyLDAvis.enable_notebook()
    vis = pyLDAvis.gensim.prepare(lda_model, bow_corpus, dic)
    return vis

In [ ]:
lda_model, bow_corpus, dic = get_lda_objects(train['text'])

In [ ]:
lda_model.show_topics()

In [ ]:
plot_lda_vis(lda_model, bow_corpus, dic)

### 2.4.3. 훈련 데이터 LDA

In [ ]:
lda_model, bow_corpus, dic = get_lda_objects(test['text'])

In [ ]:
plot_lda_vis(lda_model, bow_corpus, dic)

### 2.4.4. 감정 분석(Sentiment Analysis)

In [ ]:
from textblob import TextBlob
    
def plot_polarity_histogram(text):
    
    def _polarity(text):
        return TextBlob(text).sentiment.polarity
        
    polarity_score = text.apply(lambda x : _polarity(x))
    return polarity_score

In [ ]:
fig = plt.figure(figsize=(10,5)) # rows*cols 행렬의 i번째 subplot 생성
rows = 2
cols = 4

for i in range(8):
    polarity_score = plot_polarity_histogram(train[train['author']==i]['text'])
    ax = fig.add_subplot(rows, cols, i+1)
    ax.set_title(i)
    polarity_score.hist()
    plt.tight_layout()

### 2.4.5. 평가 자료 극성 분석(Test Data Polarity)

In [ ]:
polarity_score = plot_polarity_histogram(test['text'])
polarity_score.hist()

In [ ]:
def sentiment_vader(text, sid):
    ss = sid.polarity_scores(text)
    ss.pop('compound')
    return max(ss, key=ss.get)

def sentiment_textblob(text):
        x = TextBlob(text).sentiment.polarity
        
        if x<0:
            return 'neg'
        elif x==0:
            return 'neu'
        else:
            return 'pos'

def plot_sentiment_barchart(text, method='TextBlob'):
    if method == 'TextBlob':
        sentiment = text.map(lambda x: sentiment_textblob(x))
    elif method == 'Vader':
        nltk.download('vader_lexicon')
        sid = SentimentIntensityAnalyzer()
        sentiment = text.map(lambda x: sentiment_vader(x, sid=sid))
    else:
        raise ValueError('Textblob or Vader')
    
    return sentiment

In [ ]:
fig = plt.figure(figsize=(10,5)) # rows*cols 행렬의 i번째 subplot 생성
rows = 2
cols = 4

for i in range(8):
    sentiment = plot_sentiment_barchart(train[train['author']==i]['text'])
    ax = fig.add_subplot(rows, cols, i+1)
    #plt.bar(sentiment.value_counts().index, sentiment.value_counts())
    ax.set_title(i)
    sns.barplot(sentiment.value_counts().index, sentiment.value_counts())
    plt.tight_layout()

### 2.4.6. 평가 데이터 감정 분석(Test Data Sentiment Analysis)

In [ ]:
sentiment = plot_sentiment_barchart(test['text'])
sns.barplot(sentiment.value_counts().index, sentiment.value_counts())

### 2.4.7 개체명인식(NER : Named Entity Recognition)

In [ ]:
import spacy
from collections import  Counter
import seaborn as sns

def plot_named_entity_barchart(text):
    nlp = spacy.load("en_core_web_sm")
    
    def _get_ner(text):
        doc=nlp(text)
        return [X.label_ for X in doc.ents]
    
    ent=text.apply(lambda x : _get_ner(x))
    ent=[x for sub in ent for x in sub]
    counter=Counter(ent)
    count=counter.most_common()
    
    x,y=map(list,zip(*count))
    return x,y

In [ ]:
fig = plt.figure(figsize=(20,20)) # rows*cols 행렬의 i번째 subplot 생성
rows = 2
cols = 4

for i in range(8):
    x, y = plot_named_entity_barchart(train[train['author']==i]['text'])
    ax = fig.add_subplot(rows, cols, i+1)
    ax.set_title(i)
    sns.barplot(x=y,y=x)

### 2.4.8 사람 이름 분포

In [ ]:
import spacy
from collections import Counter
import seaborn as sns


def plot_most_common_named_entity_barchart(text, entity="PERSON"):
    nlp = spacy.load("en_core_web_sm")

    def _get_ner(text, ent):
        doc = nlp(text)
        return [X.text for X in doc.ents if X.label_ == ent]

    entity_filtered = text.apply(lambda x: _get_ner(x, entity))
    entity_filtered = [i for x in entity_filtered for i in x]

    counter = Counter(entity_filtered)
    x, y = map(list, zip(*counter.most_common(10)))
    return x, y

In [ ]:
fig = plt.figure(figsize=(20, 15))  # rows*cols 행렬의 i번째 subplot 생성
rows = 2
cols = 4

for i in range(8):
    x, y = plot_most_common_named_entity_barchart(
        train[train['author'] == i]['text'], entity="PERSON")
    ax = fig.add_subplot(rows, cols, i + 1)
    sns.barplot(y, x).set_title(i)

# 3. 데이터 임베딩

## 3.1. 임베딩에 필요한 라이브러리 및 사전 학습 데이터 가져오기

In [ ]:
import gensim
from gensim.models.keyedvectors import KeyedVectors

In [ ]:
pip install -U gensim

In [ ]:
FastText = KeyedVectors.load_word2vec_format('/content/drive/MyDrive/Novelist_Classification/Embedding/fasttext.vec')

In [ ]:
Word2Vec_model = gensim.models.KeyedVectors.load_word2vec_format(
    '/content/drive/MyDrive/Novelist_Classification/Embedding/GoogleNews-vectors-negative300.bin.gz', binary=True)

In [ ]:
#파라미터 설정
vocab_size = 63727
embedding_dim = 16
max_length = 212
padding_type='post'

In [ ]:
#tokenizer에 fit
tokenizer = Tokenizer(num_words = vocab_size) #, oov_token=oov_tok)
tokenizer.fit_on_texts(X_train)
word_index = tokenizer.word_index

In [ ]:
#데이터를 sequence로 변환해주고 padding 해줍니다.
train_sequences = tokenizer.texts_to_sequences(X_train)
# len(train_sequences)
idx = []
for i in range(len(train_sequences)):
    if len(train_sequences[i]) > 213 :
        idx.append(i)

# len(idx)
# type(X_train)
# X_train = np.delete(X_train,idx,0)
# y_train = np.delete(y_train, idx)

train = train.drop(idx)

In [ ]:
tmp = train

vocab = nltk.FreqDist(np.hstack(tmp['text']))
print('단어 집합의 크기 : {}'.format(len(vocab)))

In [ ]:
X_train = np.array([x for x in train['text']])
X_test = np.array([x for x in test['text']])
y_train = np.array([x for x in train['author']])

In [ ]:
#단어 집합 크기에 따른 단어파라미터 재설정
vocab_size = 63162

In [ ]:
#tokenizer에 fit
tokenizer = Tokenizer(num_words = vocab_size) #, oov_token=oov_tok)
tokenizer.fit_on_texts(X_train)
word_index = tokenizer.word_index

In [ ]:
#데이터를 sequence로 변환해주고 padding 해줍니다.
train_sequences = tokenizer.texts_to_sequences(X_train)

# train_sequences
train_padded = pad_sequences(train_sequences, padding=padding_type, maxlen=max_length)

# train_padded
test_sequences = tokenizer.texts_to_sequences(X_test)
test_padded = pad_sequences(test_sequences, padding=padding_type, maxlen=max_length)

## 3.2. Word2Vec

In [ ]:
# Word2Vec 임베딩 과정
W2V_embedding_matrix = np.zeros((vocab_size,300))

def get_vector(word):
    if word in Word2Vec_model:
        return Word2Vec_model[word]
    else:
        return None

In [ ]:
# print(vocab.items())
for idx, word in word_index.items():

    temp = get_vector(word)
    if temp is not None:
        W2V_embedding_matrix[idx] = temp
        
print(W2V_embedding_matrix.shape)

In [ ]:
vocab.items()

In [ ]:
word_index.items()

## 3.3. Fasttext

In [ ]:
# Fasttext 임베딩 과정
FT_embedding_matrix = np.zeros((vocab_size,100))

def get_vector(word):
    if word in FastText:
        return FastText[word]
    else:
        return None

for word, idx in word_index.items():
    temp = get_vector(word)
    if temp is not None:
        FT_embedding_matrix[idx] = temp

## 3.4. Glove

In [ ]:
# Glove 임베딩 과정
embedding_dict= dict()
f = open('/content/drive/MyDrive/Novelist_Classification/Embedding/glove.txt', encoding='utf8')

for line in f:
    word_vector = line.split()
    word = word_vector[0]
    word_vector_arr = np.asarray(word_vector[1:], dtype='float32')
    embedding_dict[word] = word_vector_arr
f.close

glove_embedding_matrix = np.zeros((vocab_size+1, 50))

for word, i in word_index.items():
    temp = embedding_dict.get(word)
    if temp is not None:
        glove_embedding_matrix[i] = temp

# 4. 모델 구축

## 4.1. Simple DNN

In [ ]:
# 간단한 DNN 모델
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(8, activation='softmax')
])

In [ ]:
# compile model
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# model summary
print(model.summary())

In [ ]:
# fit model
num_epochs = 20
history = model.fit(train_padded, y_train, 
                    epochs=num_epochs, verbose=2, 
                    validation_split=0.2)

결과에 따라 우리는 소설 작가 예측 모델을 Fasttext 임베딩과 Glove 임베딩만을 적용해보기로 했다.